In [1]:
import requests
import pandas as pd
from datetime import datetime
import time

# Coordinates and Parameters
LATITUDE = -7.0520702239386175
LONGITUDE = 110.43532807750137
TIMEZONE = "Asia/Jakarta"
API_URL = "https://archive-api.open-meteo.com/v1/archive"

# Weather Condition Mapping
def map_weather_code(code):
    """Maps WMO weather code to user's custom condition string."""
    if code == 0:
        return 'Clear'
    elif code in [1, 2]:
        return 'Partially cloudy'
    elif code in [3, 45, 48]:
        return 'Overcast'
    elif code in [51, 53, 55]:
        return 'Rain'
    elif code in [61, 63, 65]:
        return 'Rain, Overcast'
    elif code in [80, 81, 82]:
        return 'Rain, Partially cloudy'
    elif code in [95, 96, 99]:
        return 'Rain'
    else:
        return 'Unknown'

def fetch_historical_data():
    today = datetime.now().strftime("%Y-%m-%d")
    start_date = "2000-01-01"
    
    print(f"Fetching data from {start_date} to {today}...")
    
    params = {
        "latitude": LATITUDE,
        "longitude": LONGITUDE,
        "start_date": start_date,
        "end_date": today,
        "daily": ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", 
                  "relative_humidity_2m_mean", "wind_speed_10m_max", 
                  "pressure_msl_mean", "weather_code"],
        "timezone": TIMEZONE
    }
    
    try:
        response = requests.get(API_URL, params=params)
        response.raise_for_status()
        data = response.json()
        
        daily = data.get("daily", {})
        if not daily:
            print("No daily data found.")
            return

        # Create DataFrame
        df = pd.DataFrame({
            "date": daily["time"],
            "tempmax": daily["temperature_2m_max"],
            "tempmin": daily["temperature_2m_min"],
            "temp": daily["temperature_2m_mean"],
            "humidity": daily["relative_humidity_2m_mean"],
            "windspeed": daily["wind_speed_10m_max"],
            "sealevelpressure": daily["pressure_msl_mean"],
            "weather_code": daily["weather_code"]
        })
        
        # Process columns
        df["date"] = pd.to_datetime(df["date"])
        df["day"] = df["date"].dt.day
        df["month"] = df["date"].dt.month
        df["year"] = df["date"].dt.year
        
        # Map conditions
        df["conditions"] = df["weather_code"].apply(map_weather_code)
        
        # Add ID
        df["id"] = range(len(df))
        
        # Select and reorder columns
        output_columns = ["id", "day", "month", "year", "tempmax", "tempmin", "temp", 
                          "humidity", "windspeed", "sealevelpressure", "weather_code", "conditions"]
        
        final_df = df[output_columns]
        
        # Save to CSV
        output_file = "historical_data.csv"
        final_df.to_csv(output_file, index=False)
        print(f"Data successfully saved to {output_file}")
        print(f"Total records: {len(final_df)}")
        print(final_df.head())
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

fetch_historical_data()

Fetching data from 2000-01-01 to 2025-12-04...
Data successfully saved to historical_data.csv
Total records: 9470
   id  day  month  year  tempmax  tempmin  temp  humidity  windspeed  \
0   0    1      1  2000     27.5     20.8  24.1        91       12.1   
1   1    2      1  2000     27.0     21.6  23.8        93       13.0   
2   2    3      1  2000     24.9     22.1  23.3        97        8.0   
3   3    4      1  2000     27.6     21.9  24.3        89        6.1   
4   4    5      1  2000     28.9     21.3  24.2        90        5.6   

   sealevelpressure  weather_code      conditions  
0            1007.3            53            Rain  
1            1007.3            61  Rain, Overcast  
2            1007.9            61  Rain, Overcast  
3            1008.4            53            Rain  
4            1009.1            65  Rain, Overcast  
